# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import numpy as np
import pandas as pd
from sqlalchemy import create_engine
import nltk
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import fbeta_score, make_scorer
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from sklearn.multioutput import MultiOutputClassifier
import pickle
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [2]:
# load data from database
engine = create_engine('sqlite:///disaster_response.db')
df = pd.read_sql('SELECT * FROM messages_df', engine)
X = df.filter(items=['id', 'message', 'original', 'genre'])
y = df.drop(['id', 'message', 'original', 'genre', 'child_alone'], axis=1)#'child_alone' has no responses
#Mapping the '2' values in 'related' to '1' - because I consider them as a response (that is, '1')
y['related']=y['related'].map(lambda x: 1 if x == 2 else x)
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 2. Write a tokenization function to process your text data

In [3]:
def tokenize(text):
    tokens = nltk.word_tokenize(text)
    lemmatizer = nltk.WordNetLemmatizer()
    return [lemmatizer.lemmatize(w).lower().strip() for w in tokens]

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [4]:
pipeline = Pipeline([('cvect', CountVectorizer(tokenizer = tokenize)),
                     ('tfidf', TfidfTransformer()),
                     ('clf', RandomForestClassifier())
                     ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y)
pipeline.fit(X_train['message'], y_train)

Pipeline(memory=None,
     steps=[('cvect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        stri...n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [6]:
y_pred_test = pipeline.predict(X_test['message'])
y_pred_train = pipeline.predict(X_train['message'])
print('-----------------------------On testing data set----------------------------------------')
print(classification_report(y_test.values, y_pred_test, target_names=y.columns.values))
print('-----------------------------On training data set----------------------------------------')
print('\n',classification_report(y_train.values, y_pred_train, target_names=y.columns.values))

-----------------------------On testing data set----------------------------------------
                        precision    recall  f1-score   support

               related       0.85      0.90      0.87      5073
               request       0.85      0.40      0.54      1135
                 offer       0.00      0.00      0.00        34
           aid_related       0.78      0.43      0.55      2778
          medical_help       0.62      0.02      0.03       530
      medical_products       0.74      0.05      0.09       340
     search_and_rescue       0.67      0.01      0.02       161
              security       0.00      0.00      0.00       109
              military       0.00      0.00      0.00       243
                 water       0.87      0.22      0.36       403
                  food       0.85      0.24      0.38       750
               shelter       0.83      0.09      0.16       591
              clothing       0.75      0.08      0.15       106
              

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [7]:
parameters = {'clf__max_depth': [10, 20, None],
              'clf__min_samples_leaf': [1, 2, 4],
              'clf__min_samples_split': [2, 5, 10],
              'clf__n_estimators': [10, 20, 40]}

cv = GridSearchCV(pipeline, param_grid=parameters, scoring='f1_micro', verbose=1, n_jobs=-1)
cv.fit(X_train['message'], y_train)

Fitting 3 folds for each of 81 candidates, totalling 243 fits


[Parallel(n_jobs=-1)]: Done 243 out of 243 | elapsed: 100.0min finished


GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('cvect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        stri...n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))]),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'clf__max_depth': [10, 20, None], 'clf__min_samples_leaf': [1, 2, 4], 'clf__min_samples_split': [2, 5, 10], 'clf__n_estimators': [10, 20, 40]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='f1_micro', verbose=1)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [8]:
y_pred_test = cv.predict(X_test['message'])
y_pred_train = cv.predict(X_train['message'])
print('--------------------------------Testing Data-----------------------------------------------')
print(classification_report(y_test.values, y_pred_test, target_names=y.columns.values))
print('--------------------------------Traning Data-----------------------------------------------')
print('\n',classification_report(y_train.values, y_pred_train, target_names=y.columns.values))

--------------------------------Testing Data-----------------------------------------------
                        precision    recall  f1-score   support

               related       0.83      0.95      0.89      5073
               request       0.85      0.42      0.56      1135
                 offer       0.00      0.00      0.00        34
           aid_related       0.81      0.52      0.63      2778
          medical_help       0.67      0.01      0.02       530
      medical_products       0.71      0.03      0.06       340
     search_and_rescue       0.00      0.00      0.00       161
              security       0.00      0.00      0.00       109
              military       1.00      0.00      0.01       243
                 water       0.89      0.21      0.34       403
                  food       0.88      0.29      0.44       750
               shelter       0.87      0.11      0.20       591
              clothing       0.50      0.01      0.02       106
           

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [9]:
pipeline2 = Pipeline([('cvect', CountVectorizer(tokenizer = tokenize)),
                     ('tfidf', TfidfTransformer()),
                     ('clf', GradientBoostingClassifier())
                     ])


In [10]:
X_train2, X_test2, y_train2, y_test2 = train_test_split(X, y)
pipeline.fit(X_train2['message'], y_train2)

Pipeline(memory=None,
     steps=[('cvect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        stri...n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))])

In [11]:
y_pred_test2 = pipeline.predict(X_test2['message'])
y_pred_train2 = pipeline.predict(X_train2['message'])
print('-----------------------------On testing data set----------------------------------------')
print(classification_report(y_test2.values, y_pred_test2, target_names=y.columns.values))
print('-----------------------------On training data set----------------------------------------')
print('\n',classification_report(y_train2.values, y_pred_train2, target_names=y.columns.values))

-----------------------------On testing data set----------------------------------------
                        precision    recall  f1-score   support

               related       0.83      0.90      0.87      4995
               request       0.80      0.37      0.51      1096
                 offer       0.00      0.00      0.00        30
           aid_related       0.79      0.42      0.55      2709
          medical_help       0.48      0.02      0.04       497
      medical_products       0.92      0.04      0.08       302
     search_and_rescue       0.67      0.01      0.02       200
              security       0.00      0.00      0.00       117
              military       0.00      0.00      0.00       213
                 water       0.83      0.09      0.16       419
                  food       0.85      0.20      0.32       734
               shelter       0.78      0.07      0.12       579
              clothing       0.83      0.05      0.09       100
              

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 9. Export your model as a pickle file

In [12]:
cv.best_params_

{'clf__max_depth': None,
 'clf__min_samples_leaf': 1,
 'clf__min_samples_split': 10,
 'clf__n_estimators': 20}

In [14]:
m = pickle.dumps('clf')

In [15]:
pickle.dump(cv, open('disaster_model.sav', 'wb'))

In [18]:
loaded_model = pickle.load(open('disaster_model.sav', 'rb'))

print(loaded_model)

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('cvect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        stri...n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))]),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'clf__max_depth': [10, 20, None], 'clf__min_samples_leaf': [1, 2, 4], 'clf__min_samples_split': [2, 5, 10], 'clf__n_estimators': [10, 20, 40]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='f1_micro', verbose=1)


In [19]:
y_pred_test = loaded_model.predict(X_test['message'])
y_pred_train = loaded_model.predict(X_train['message'])
print('--------------------------------Testing Data-----------------------------------------------')
print(classification_report(y_test.values, y_pred_test, target_names=y.columns.values))
print('--------------------------------Traning Data-----------------------------------------------')
print('\n',classification_report(y_train.values, y_pred_train, target_names=y.columns.values))

--------------------------------Testing Data-----------------------------------------------
                        precision    recall  f1-score   support

               related       0.83      0.95      0.89      5073
               request       0.85      0.42      0.56      1135
                 offer       0.00      0.00      0.00        34
           aid_related       0.81      0.52      0.63      2778
          medical_help       0.67      0.01      0.02       530
      medical_products       0.71      0.03      0.06       340
     search_and_rescue       0.00      0.00      0.00       161
              security       0.00      0.00      0.00       109
              military       1.00      0.00      0.01       243
                 water       0.89      0.21      0.34       403
                  food       0.88      0.29      0.44       750
               shelter       0.87      0.11      0.20       591
              clothing       0.50      0.01      0.02       106
           

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.